Initially we had created images without caring about the colormap of each image<br>
And then we identified which matFile consisted of invalid data.<br>
This included the automated and manual check we had to do on the images. <br><br>

To reduce the computation of checking the criteria of valid or invalid data.<br>
We will iterate through the csv files which is collection of valid images<br>
And create a json of .mat files consisting of valid frequencies.

In [1]:
import re
import os
import time
import json
import math
import scipy.io
import numpy as np
import pandas as pd

In [2]:
imgFreq = {}
tbsVal = {}
fList = ['19H','19V','19','22V','37V','37H','37','91H','91V','91','150H','183_1H','183_3H','183_7H']
rList = ['ATL','CPAC','EPAC','IO','SHEM','WPAC']
for r in rList:
    tbsVal[r] = {}
    imgFreq[r] = {}
    for f in fList:
        tbsVal[r][f] = [[],[]]
        imgFreq[r][f] = pd.read_csv("..\\ImagesPerFreq\\"+r+"_"+f+'.csv')

### Code to find out Max Min Brightness temperatures

In [3]:
valid_matfile_and_best_track = json.load( open( "..\\valid_matfiles_and_best_track.json" ) )

tbsVal will contain list of all max and min tbs values for each unique combination of region and freq

In [4]:
class FindMaxMinTbs:
    global tbsVal
    def __init__(self, matpath):
        self.mMatPath = matpath
        
        self.__mFreq_to_swath = {}
        self.__mFreq_to_swath['19']  = [0,[2.400, 1.400]] 
        self.__mFreq_to_swath['19V'] = [0,0]
        self.__mFreq_to_swath['19H'] = [0,1]
        self.__mFreq_to_swath['22V'] = [0,2]

        self.__mFreq_to_swath['37']  = [1,[2.150, 1.150]]
        self.__mFreq_to_swath['37V'] = [1,0]
        self.__mFreq_to_swath['37H'] = [1,1]

        self.__mFreq_to_swath['91']  = [3,[1.751, 0.751]]
        self.__mFreq_to_swath['91V'] = [3,0] 
        self.__mFreq_to_swath['91H'] = [3,1]

        self.__mFreq_to_swath['150H'] = [2,0]
        self.__mFreq_to_swath['183_1H'] = [2,1]
        self.__mFreq_to_swath['183_3H'] = [2,2]
        self.__mFreq_to_swath['183_7H'] = [2,3]
     
    def PCT_Function(self,v,h,val):
        return val[0]*v - val[1]*h
    
    def ReadMatFile(self,region,matFile,valid_freq):
        try:
            mat = scipy.io.loadmat(self.mMatPath+matFile)
        except:
            msg = "Error Reading File: " + str( self.mMatPath+matFile ) 
            print(msg)
            #self.__mLog.error( msg )
       
        swaths = mat["passData"][0][0]
        
        for freq in valid_freq:
            swathList = self.__mFreq_to_swath[freq]
            
            swath_data = swaths[ swathList[0] ]
            lat = swath_data[0][0][1]
            lon = swath_data[0][0][2]
            channel = swath_data[0][0][3]
            
             # Calculating PCT values for specific frequencies
            if freq == '19' or freq=='37' or freq=='91':
                tbs = self.PCT_Function( channel[0], channel[1], swathList[1] )
            else:
                tbs = channel[swathList[1]]
            
            if np.any(tbs<320) and np.any(tbs>0):
                tbsVal[region][freq][0].append(np.amin(tbs))
                tbsVal[region][freq][1].append(np.amax(tbs))

Iterate over all mat files and find out Max and Min values of each frequencies

In [6]:
s = time.time()
for region,v1 in valid_matfile_and_best_track.items():
    for year,v2 in v1.items():
        for stormNo,stormDict in v2.items():
            for f_,files in stormDict.items():
                if f_ == "BestTrack":
                    continue
                rootDirOfMatFile = files[0]
                matFiles = files[1]
                ftbs = FindMaxMinTbs(rootDirOfMatFile)
                for filename,valid_freq in matFiles.items():
                    ftbs.ReadMatFile(region, filename,valid_freq)
print("Total Time taken: "+str( (time.time()-s)/60 )+" mins" )

Total Time taken: 26.787337613105773 mins


### Analysis of the Max and Min Tbs Values found after above process

In [5]:
import matplotlib.pyplot as plt
import statistics
def MakeDir(path):
    if os.path.isdir(path) == False:
        os.mkdir(path)

In [7]:
tbsVal["All"] = {}
for f in fList:
    tbsVal["All"][f] =[[],[]]
    for i in range(2):
        for r in rList:
            tbsVal["All"][f][i].extend(tbsVal[r][f][i])

In [8]:
# Create Histogram Graphs
path = os.getcwd()+"\\MaxMinTbs\\"
for i in range(2):
    mpath = path+str(i)
    MakeDir(mpath)
    for f in fList:
        for r in rList + ["All"]:
            fpath=mpath+"\\"+f
            MakeDir(fpath)
            plt.hist( tbsVal[r][f][i])
            meanVal = "{:.2f}".format(np.mean(tbsVal[r][f][i]))
            stdVal = "{:.2f}".format(np.std(tbsVal[r][f][i]))
            plt.savefig(fpath+"\\"+r+"_"+meanVal+"_"+stdVal+".png")
            plt.close()

In [9]:
# Create Tables
dic_tbs_val = [ [min,{}], [max,{}] ]
                 
for i in range(2):
    for r in rList:
        dic_tbs_val[i][1][r] = {}
        for f in fList:
            dic_tbs_val[i][1][r][f] = [ dic_tbs_val[i][0](tbsVal[r][f][i]), np.mean(tbsVal[r][f][i]), np.std(tbsVal[r][f][i]) ]

In [10]:
t1 = pd.DataFrame(dic_tbs_val[0][1])
t2 = pd.DataFrame(dic_tbs_val[1][1])

In [11]:
t1.to_csv(os.getcwd()+"\\minimum.csv")
t2.to_csv(os.getcwd()+"\\maximum.csv")